In [1]:
# to download the dataset
!gdown --id 1K5Ai6robPRva0eKDOrPLRvQ_6gpTIaDe
# to unzip the file
!unzip finalize_dogs_vs_cats.zip

Downloading...
From: https://drive.google.com/uc?id=1K5Ai6robPRva0eKDOrPLRvQ_6gpTIaDe
To: C:\Repositories\MachineLearning\week15\finalize_dogs_vs_cats.zip

  0%|          | 0.00/428M [00:00<?, ?B/s]
  0%|          | 524k/428M [00:00<03:13, 2.21MB/s]
  0%|          | 1.05M/428M [00:00<02:11, 3.24MB/s]
  1%|          | 2.62M/428M [00:00<01:00, 7.07MB/s]
  1%|1         | 4.72M/428M [00:00<00:39, 10.6MB/s]
  2%|1         | 6.82M/428M [00:00<00:47, 8.78MB/s]
  2%|1         | 7.86M/428M [00:00<00:47, 8.77MB/s]
  2%|2         | 9.96M/428M [00:01<00:36, 11.4MB/s]
  3%|2         | 12.6M/428M [00:01<00:29, 14.3MB/s]
  4%|3         | 15.2M/428M [00:01<00:24, 16.8MB/s]
  4%|4         | 18.4M/428M [00:01<00:21, 19.0MB/s]
  5%|5         | 21.5M/428M [00:01<00:18, 21.7MB/s]
  6%|5         | 24.1M/428M [00:01<00:17, 22.5MB/s]
  6%|6         | 27.8M/428M [00:01<00:15, 25.5MB/s]
  7%|7         | 30.9M/428M [00:01<00:15, 26.1MB/s]
  8%|8         | 34.6M/428M [00:02<00:13, 28.6MB/s]
  9%|8         | 38.3M

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define model
def define_model():
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('finalize_dogs_vs_cats/',
                                           class_mode='binary', batch_size=64, target_size=(224,224))
    # fit model
    model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=5, verbose=1)
    # save model
    model.save('final_model.h5')
    
# entry point, run the test harness
run_test_harness()

Found 18697 images belonging to 2 classes.
Epoch 1/5
293/293 [==============================] - 63s 212ms/step - loss: 0.2283 - accuracy: 0.9659
Epoch 2/5
293/293 [==============================] - 60s 205ms/step - loss: 0.0242 - accuracy: 0.9913
Epoch 3/5
293/293 [==============================] - 59s 200ms/step - loss: 0.0059 - accuracy: 0.9989
Epoch 4/5
293/293 [==============================] - 59s 201ms/step - loss: 0.0015 - accuracy: 0.9998
Epoch 5/5
293/293 [==============================] - 59s 201ms/step - loss: 7.8414e-04 - accuracy: 0.9999


In [6]:
from keras.models import load_model
import tensorflow as tf

# load saved model
model = load_model('final_model.h5')

# convert Keras model to TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save converted model
with open('dogvscat_class_model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\wjsdu\AppData\Local\Temp\tmp0nttatfj\assets
